In [1]:
import pandas as pd
import gc

# Change folder to location of output of create_data.py
folder = '/Users/BharathiSrinivasan/Documents/GitHub/Thesis/'
df = pd.read_csv(folder + 'merged_data.csv')

################ Product Features ############################################################

sub1 = df.sort_values(['product_id'],ascending=True)

# 'product_id_orders' indicates the popularity of a product
sub2 = sub1.join(sub1.groupby('product_id')['product_id'].size(), on='product_id', rsuffix='_orders')

# 'reordered_total' indicates how many times a product was reordered
sub3 = sub2.join(sub2.groupby('product_id')['reordered'].sum(), on='product_id', rsuffix='_total')

prd = sub3[['user_id','product_id', 'product_id_orders','reordered_total']]

del sub1,sub2,sub3
gc.collect()

print('Product related features created!')



Product related features created!


In [2]:
############# User Features ##################################################################

# Total orders by user
users = df.groupby(['user_id'])['order_number'].max().to_frame('user_orders')

# Not ideal - but an indidcation of customer's history with the business
users['user_period'] = df.groupby(['user_id'])['days_since_prior_order'].sum()

# Average time taken by a user before returning for a purchase
users['avg_time_to_order'] = df.groupby(['user_id'])['days_since_prior_order'].mean()

# Total unique products ordered by a customer
users['user_distinct_products'] = df.groupby(['user_id'])['product_id'].nunique()
 
# Total products ordered by a customer
users['total_products'] = df.groupby(['user_id'])['product_id'].size()

# the average basket size of the user
users['user_average_basket'] = users['total_products'] / users['user_orders']

print('User related features created!')

User related features created!


In [3]:
########### User-Product Features ###########################################################

# Product Reorder Frequency indicates a user's preference for a particular product
df['prd_reorder_freq'] = df[df['reordered']==1].groupby(['user_id', 'product_id']).cumcount()+1

# Average position of the product in a user's cart
df1 = df.groupby(['user_id', 'product_id'])['add_to_cart_order'].mean().to_frame('average_cart_position')

print('User-Product Features created!')


User-Product Features created!


In [4]:
users.to_csv('users_features.csv',index = False, header = True, sep = ',', encoding = 'utf-8')

In [5]:
prd.to_csv('prd_features.csv',index = False, header = True, sep = ',', encoding = 'utf-8')

In [6]:
u = pd.read_csv('users_features.csv')

In [7]:
u.head()

,user_orders,user_period,avg_time_to_order,user_distinct_products,total_products,user_average_basket
0,11,1304.820763,18.640297,19,70,6.363636
1,15,3983.733984,17.627142,121,226,15.066667
2,12,1009.641526,11.473199,33,88,7.333333
3,5,260.456611,14.469812,17,18,3.600000
4,5,556.005679,12.087080,28,46,9.200000


In [8]:
########### Merge all features to create final dataframe ####################################
print('Merging features into dataframe')
df1 = df1.reset_index()
df2 = pd.merge(df, df1, how='left', on=['user_id', 'product_id'])

Merging features into dataframe


In [9]:
users = users.reset_index()
df3 = pd.merge(df2, users, how='left', on='user_id')
print('users merged')

users merged


In [13]:
import dask.dataframe as dd

In [11]:
df3.to_csv('temp.csv',index = False, header = True, sep = ',', encoding = 'utf-8')

In [14]:
data_1=dd.read_csv('prd_features.csv',dtype="object",encoding='cp1252')

In [15]:
data_2=dd.read_csv('temp.csv',dtype="object",encoding='cp1252')

In [16]:
final_1=dd.merge(data_1,data_2,left_on='product_id',right_on='product_id',how="left")

In [22]:
dd.to_csv(final_1,'final_data.csv',index=False,low_memory=False,sep = ',', encoding = 'utf8',errors='ignore')

UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 23: character maps to <undefined>

In [12]:
prd = prd.reset_index()
df_final = dd.merge(df3, prd, how ='left', on='product_id')
print('products merged')

AttributeError: module 'dask' has no attribute 'merge'

In [ ]:
df_final.drop(['Unnamed: 0', 'user_id_x','index_x','level_0','index_y','user_id_y'], axis=1, inplace=True)

In [23]:
# Export dataset for future use
print('Final dataset writing to csv')

outfile = open(folder+'final_data.csv', 'wb')
final_1.to_csv('final_data.csv',index = False, header = True, sep = ',', encoding = 'utf-8')
outfile.close()

Final dataset writing to csv


UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 23: character maps to <undefined>